# include library

* [https://github.com/cbeust/klaxon](https://github.com/cbeust/klaxon)
* [https://www.baeldung.com/kotlin-json-klaxson](https://www.baeldung.com/kotlin-json-klaxson)
* [https://www.kotlinresources.com/library/klaxon/](https://www.kotlinresources.com/library/klaxon/)
* 

In [2]:
%use klaxon

In [2]:
Klaxon()

com.beust.klaxon.Klaxon@2fb79723

# General Parse

In [3]:
class Person(val name: String, var age: Int = 23)

val result = Klaxon()
    .parse<Person>(
        """
{
  "name": "John Smith",
}
"""
    )

println("${result?.name}, ${result?.age}")
assert(result?.name == "John Smith")
assert(result?.age == 23)

John Smith, 23


# Json annotation

## mapping between Json document and Kotlin object via use the name attribute

In [4]:
data class Person(
    @Json(name = "the_name")
    val name: String,
    val age: Int
)

val result = Klaxon()
    .parse<Person>(
        """
{
  "the_name": "John Smith",
  "age": 23
}
"""
    )
println("${result?.name}, ${result?.age}")
assert(result?.name == "John Smith")
assert(result?.age == 23)

John Smith, 23


## ignore attribute in parsing process

In [5]:
class Ignored(val name:String, val age:Int){
    @Json(ignored = true)
    val actualName: String = "AAA"
    @Json(ignored = false)
    val actualAge : Int = 10
}

val result2 = Klaxon()
    .parse<Ignored>(
        """{
            "name":"John Smith",
            "age":23,
            "actualName":"BBB",
            "actualAge": 50
            }
        """
    )

println("${result2?.name}, ${result2?.age}")
// actualName的值在parsing後沒有被置換掉，actualAge的值被置換了
println("${result2?.actualName}, ${result2?.actualAge}")


John Smith, 23
AAA, 50


## display order

In [6]:
class Data(
    @Json(index = 1) val id: String,
    @Json(index = 2) val name: String
)
println(Klaxon().toJsonString(Data("A123", "Kotlin")))

class Data2(
    @Json(index = 2) val id: String,
    @Json(index = 1) val name: String
)
println(Klaxon().toJsonString(Data2("A123", "Kotlin")))

{"id" : "A123", "name" : "Kotlin"}
{"name" : "Kotlin", "id" : "A123"}


# Streaming API
* allows our code to process JSON values while it is still reading

In [6]:
import java.io.StringReader

val jsonArray = """
    [
        { "name" : "HDD", "capacityInGb" : 512 },
        { "name" : "RAM", "capacityInGb" : 16 }
    ]"""

val jsonObject = """
    {
      "name": "John Smith",
      "age": 20
    }        
""".trimIndent()

data class ProductData(val name: String, val capacityInGb: Int)
//    data class Person(val name: String, val age: Int)

val expectedArray = arrayListOf(
    ProductData("HDD", 512),
    ProductData("RAM", 16)
)
val klaxon = Klaxon()
val productArray = arrayListOf<ProductData>()
JsonReader(StringReader(jsonArray)).use { reader ->
    reader.beginArray {
        while (reader.hasNext()) {
            val product = klaxon.parse<ProductData>(reader)
            productArray.add(product!!)
        }
    }
}

JsonReader(StringReader(jsonObject)).use { reader ->
    reader.beginObject {
        while (reader.hasNext()) {
            when(reader.nextName()){
                "name" -> println("name:${reader.nextString()}")
                "age" -> println("age:${reader.nextInt()}")
            }
        }
    }
}

//    assertTrue(productArray.size == expectedArray.size)
// assertThat(productArray).hasSize(2).isEqualTo(expectedArray)
productArray.forEach {
    println("name:${it.name}, capacity:${it.capacityInGb}")
}

name:John Smith
age:20
name:HDD, capacity:512
name:RAM, capacity:16


# define path matchers to locate specific entries in our documents

In [18]:
import java.util.regex.Pattern
val pathMatcher = object : PathMatcher {
    override fun pathMatches(path: String) = Pattern.matches(".*inventory.*disks.*type.*", path)

    override fun onMatch(path: String, value: Any) {
//         println(path)
        when (path) {
            "$.inventory.disks[0].type"
            -> println(value == "HDD")
            "$.inventory.disks[1].type"
            -> println(value == "SDD")
        }
    }
}

val jsonString = """
    {
        "inventory" : {
            "disks" : [
                {
                    "type" : "HDD",
                    "sizeInGb" : 1000
                },
                {
                    "type" : "SDD",
                    "sizeInGb" : 512
                }
            ]
        }
    }

"""
val result =
    Klaxon().pathMatcher(pathMatcher)
        .parseJsonObject(StringReader(jsonString))


true
true


# process JSON documents like a Map or a List
## JsonObject like Map

In [12]:
val jsonString = StringBuilder(
    """
        {
            "name" : "HDD",
            "capacityInGb" : 512,
            "sizeInInch" : 2.5
        }
    """
)
val parser = Parser()
val json = parser.parse(jsonString) as JsonObject

json.forEach { k, v ->
    println("$k:$v")
}

name:HDD
capacityInGb:512
sizeInInch:2.5


## JsonArray like List

In [10]:
val jsonString = StringBuilder(
    """
[
    { "name" : "SDD" },
    { "madeIn" : "Taiwan" },
    { "warrantyInYears" : 5 }
]
"""
)
val parser = Parser()
val json = parser.parse(jsonString) as JsonArray<JsonObject>

json.forEach {
    it.forEach { k, v ->
        println("$k: $v")
    }
}

name: SDD
madeIn: Taiwan
warrantyInYears: 5
